This script trains GPT 3 using it's few-short-learning ability to add inline-comments to a given code snippet.
Insert any python code snippet as prompt and the model will add inline comments to your code that explain the corresponding lines.

In [ ]:
from gpt import GPT, Example
import openai

# insert your key that is able to call the davinci-codex engine
openai.api_key = 'YOUR_API_KEY'

In [ ]:
# set engine, temperature and max_tokens
gpt3 = GPT(engine="davinci-codex", temperature=0.1, max_tokens=250)

In [ ]:
# add few-shot-learning examples to train GPT3 to the use case
gpt3.add_example(Example(
"""
def greet(*names):
	i=0
	print('Hello ', end='')
	while len(names) > i:
		print(names[i], end=', ')
		i+=1
    
Return the above function but add inline comments that explain what the function does:
"""
,
"""
def greet(*names):
	i=0
	print('Hello ', end='')
	while len(names) > i:   # loop through names
		print(names[i], end=', ')
		i+=1 
"""))

gpt3.add_example(Example(
"""
 def insert(self, user):
	cursor = self._cnx.cursor()
	cursor.execute("SELECT MAX(id) AS maxid FROM user ")
	ins = cursor.fetchall()

	for (maxid) in ins:
		if maxid[0] is not None:
			user.set_id(maxid[0] + 1)
		else:
			user.set_id(1)

	template = "INSERT INTO user (id, name) VALUES (%s,%s)"
	vals = (user.get_id(), user.get_name())
	cursor.execute(template, vals)

	self._cnx.commit()
	cursor.close()

	return user
    
Return the above function but add inline comments that explain what the function does:
"""
,
"""
def insert(self, user):
	cursor = self._cnx.cursor()
	cursor.execute("SELECT MAX(id) AS maxid FROM user ")   # execute SQL statement to get highest id from user table
	ins = cursor.fetchall()

	for (maxid) in ins:
		if maxid[0] is not None:    # if maxid is found, set id of new user to maxid +1  
			user.set_id(maxid[0] + 1)
		else:                       # if no maxid is found, set id of new user to 1
			user.set_id(1)

	template = "INSERT INTO user (id, name) VALUES (%s,%s)"   # declare SQL statement to insert id and name 
	                                                          # of the new user into the user table
	vals = (user.get_id(), user.get_name())
	cursor.execute(template, vals)     # execute SQL statement with values from user object

	self._cnx.commit()
	cursor.close()

	return user
"""))

gpt3.add_example(Example(
"""
def isValidChain(self):
		for i in range(1, len(self.chain)):
			b1 = self.chain[i-1]
			b2 = self.chain[i]

			if not b2.hasValidTransactions():
				print("error 3")
				return False

			if b2.hash != b2.calculateHash():
				print("error 4")
				return False

			if b2.prev != b1.hash:
				print("error 5")
				return False
		return True
    
Return the above function but add inline comments that explain what the function does:
"""
,
"""
def isValidChain(self):
		for i in range(1, len(self.chain)):   # loop through blockchain
			b1 = self.chain[i-1]
			b2 = self.chain[i]

			if not b2.hasValidTransactions():    # check if block2 has valid transactions
				print("error 3")
				return False

			if b2.hash != b2.calculateHash():    # check if the hash of block2 is correct
				print("error 4")
				return False

			if b2.prev != b1.hash:    # check if the previous hash of block2 maches the hash of block1
				print("error 5")
				return False
		return True
"""))

In [ ]:
# declare some example prompts to test the use case 
prompt1 = """
def getBalance(self, person):
    balance = 0; 
    for i in range(1, len(self.chain)):
        block = self.chain[i]
        try:
            for j in range(0, len(block.transactions)):
                transaction = block.transactions[j]
                if(transaction.sender == person):
                    balance -= transaction.amt
                if(transaction.reciever == person):
                    balance += transaction.amt
        except AttributeError:
            print("no transaction")
    return balance + 100
"""
prompt2 = """
def isValidTransaction(self):
    if(self.hash != self.calculateHash()):
        return False
    if(self.sender == self.reciever):
        return False
    if(self.sender == "Miner Rewards"):
        #security : unfinished
        return True
    if not self.signature or len(self.signature) == 0:
        print("No Signature!")
        return False
    return True
"""
prompt3 = """
def delete(self, user):
    cursor = self._cnx.cursor()

    template = "DELETE FROM user WHERE id={}".format(user.get_id())
    cursor.execute(template)

    self._cnx.commit()
    cursor.close()
"""

In [ ]:
# generate completion response from GPT3
# insert any prompt declared above or a new one
output = gpt3.submit_request(prompt1)

# print the response
print(output.choices[0].text)